![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Implicit Recommendation from ECommerce Data

Some of the material for this work is based on [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/) by Jesse Steinweg Woods. This tutorial includes an implementation of the Alternating Least Squares algorithm and some other useful functions (like the area under the curve calculation). Other parts of the tutorial are based on a previous version of the Implicit library and had to be reworked.

The dataset used for this work is from Kaggle [Vipin Kumar Transaction Data](https://www.kaggle.com/vipin20/transaction-data):

## Context

This is a item purchased transactions data. It has 8 columns.
This data makes you familer with transactions data.

## Content

Data description is :-

* UserId -It is a unique ID for all User Id
* TransactionId -It contains unique Transactions ID
* TransactionTime -It contains Transaction Time
* ItemCode -It contains item code that item will be purchased
* ItemDescription -It contains Item description
* NumberOfItemPurchased -It contains total number of items Purchased
* CostPerltem -Cost per item Purchased
* Country -Country where item purchased


# Global Imports

In [1]:
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import implicit
import scipy
from sklearn import metrics
from pandas.api.types import CategoricalDtype
import wandb

In [2]:
%run Common-Functions.ipynb

## Hyperparameter Tuning with Weights & Biases


In [3]:
sweep_config = {
    "name": "vipin20-sweep",
    "method": "bayes",  # grid, random
    "metric": {"name": "prediction_auc", "goal": "maximize"},
    "parameters": {
        "percent_test": {"min":0.1, "max":0.3},
        "alpha": {"min":1, "max":50 },
        "regularization": {"min":0.001, "max":.3},
        "iterations": {"min":20, "max":100}
    },
}

sweep_id = wandb.sweep(sweep_config, project="vipin20")

def sweep():

    # Initialize a new wandb run
    with wandb.init() as run:

        transactions = pd.read_pickle('../data/interim/vipin20/transactions.gz')

        transaction_list = list(np.sort(transactions.TransactionId.unique())) # Get our unique customers
        item_list = list(transactions.ItemCode.unique()) # Get our unique products that were purchased
        quantity_list = list(transactions.NumberOfItemsPurchased) # All of our purchases

        cols = transactions.TransactionId.astype(CategoricalDtype(categories=transaction_list, ordered=True)).cat.codes 
        # Get the associated row indices
        rows = transactions.ItemCode.astype(CategoricalDtype(categories=item_list, ordered=True)).cat.codes 
        # Get the associated column indices
        purchases_sparse = scipy.sparse.csr_matrix((quantity_list, (rows, cols)), shape=(len(item_list), len(transaction_list)))

        product_train, product_test, products_altered, transactions_altered = make_train(purchases_sparse, pct_test = wandb.config['percent_test'])

        model = implicit.als.AlternatingLeastSquares(factors=64,
                                        regularization=wandb.config['regularization'],
                                        iterations=wandb.config['iterations'])

        alpha = wandb.config['alpha']                                                                  
        model.fit((product_train * alpha).astype('double'))

        user_vecs = model.user_factors
        item_vecs = model.item_factors

        test, popular = calc_mean_auc(product_train, products_altered, 
                      [scipy.sparse.csr_matrix(item_vecs), scipy.sparse.csr_matrix(user_vecs.T)], product_test)
        print('Logging Test Value:',test)
        wandb.log({
            'prediction_auc': test
        })


wandb.agent(sweep_id, function=sweep, count=100)

Create sweep with ID: b1xepl0k
Sweep URL: https://wandb.ai/jperiodlangley/vipin20/sweeps/b1xepl0k


wandb: Agent Starting Run: n2g3m49y with config:
wandb: 	alpha: 6
wandb: 	iterations: 27
wandb: 	percent_test: 0.29996418164015204
wandb: 	regularization: 0.18367796470780554
wandb: Currently logged in as: jperiodlangley (use `wandb login --relogin` to force relogin)


  0%|          | 0/27 [00:00<?, ?it/s]

Logging Test Value: 0.8540146483821729


prediction_auc,0.85401
_runtime,90
_timestamp,1629485764
_step,0


...

  0%|          | 0/63 [00:00<?, ?it/s]

Logging Test Value: 0.8887782510120875


prediction_auc,0.88878
_runtime,143
_timestamp,1629488476
_step,0


prediction_auc,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
